# Auswertung Pumpwerke
Arbeitsfluss:
1. Liest Pumpwerkdaten von `Daten_Pumpwerken`
2. Liest Regendaten von `Regendaten`

## Imports

In [ ]:
import math
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime as dt
from hbt_tools import utils_hbt as uh
from pathlib import Path

## Functions

In [ ]:
# Poleni
def Q_poleni(mu, L, h_w):
    return 2/3 * mu * math.sqrt(2*9.81) * L * h_w**(3/2)

## Settings

In [ ]:
path_datafolder=Path(r"Q:\Projekte\10000-\10208.10 Tangente Zug"
    r"\05 Berechnungen Grundlagen\Pumpwerken\Daten_Pumpwerken")
path_rain_agrometeo = Path(r"Q:\Projekte\10000-\10208.10 Tangente Zug"
    r"\05 Berechnungen Grundlagen\Regendaten\data_Baar.csv")
path_rain_meteoschweiz = Path(r"Q:\Projekte\10000-\10208.10 Tangente Zug"
    r"\05 Berechnungen Grundlagen\Regendaten\Cham Niederschlag"
    r"\CHZ_Niederschlagsumme.csv")

save_dir = Path(r"Q:\Projekte\10000-\10208.10 Tangente Zug"
    r"\05 Berechnungen Grundlagen")

pw_info = {
    'Pumpwerk Sternen': {
        'h_entl': 2.90, # m, 431.60 - 428.70
        'B_entl': 2.0 # m
    },
    'Pumpwerk UF Zugerstrasse': {
        'h_entl': None, # keine Entlastung
    },
    'Pumpwerk Neufeld': {
        'h_entl': None, # keine Entlastung
    },
    'Pumpwerk Südstrasse': {
        'h_entl': 3.15, # m, 428.75 - 425.60
        'B_entl': 2.0 # m
    },
    'Pumpwerk SABA': {
        'h_entl': 1.92, # m, 427.75 - 425.83
        'B_entl': 2.0 # m
    }
}

## Read data

### Pumpwerkdaten

In [ ]:
dt_format = '%d.%m.%Y %H:%M'
date_parser = lambda x: pd.to_datetime(x, format=dt_format)
paths_data = list(path_datafolder.glob("*.csv"))
df_pw = pd.DataFrame(columns=["Beschrieb","Zeitstempel","Wert"])

for i, path_data in enumerate(paths_data):
    df_temp = pd.read_csv(path_data, delimiter=';',
        names=["Beschrieb","Zeitstempel","Wert"], parse_dates=[1],
        date_parser=date_parser)
    df_pw = pd.concat([df_pw, df_temp])

df_pw.set_index('Zeitstempel', inplace=True)

sel = [
    'Pumpwerk Südstrasse Niveaumessung 1 Messwert 4-20mA -180B1',
    'Pumpwerk Südstrasse Niveaumessung 2 Messwert 4-20mA -181B1'
]
condition = (df_pw['Beschrieb']==sel[0]) | (df_pw['Beschrieb']==sel[1])
df_pw['Wert'].loc[condition] += 1.0 # 1.0m = Pumpen aus

### Regendaten

In [ ]:
ts_rain_am = uh.read_raindata_agrometeo(path_rain_agrometeo,
    resolution='10min')
ts_rain_ms = uh.read_raindata_meteoschweiz(path_rain_meteoschweiz)

## Analysis

In [ ]:
df_betrieb = df_pw[df_pw['Beschrieb'].str.contains('Betrieb')]
df_niveau = df_pw[df_pw['Beschrieb'].str.contains('Niveau')]

df_betrieb['Wert'] = df_betrieb['Wert'].astype(bool)
df_betrieb.sort_index(inplace=True)

df_betrieb2 = df_betrieb.iloc[1:,:]
df_betrieb2['Wert'] = ~df_betrieb2['Wert']
df_betrieb2.index = df_betrieb2.index - pd.Timedelta(seconds=1)
df_betrieb = pd.concat((df_betrieb, df_betrieb2))
df_betrieb.sort_index(inplace=True)
df_betrieb['Wert'] = df_betrieb['Wert'].astype(int)

### Überlaufmengen


In [ ]:
dt_format = '%d.%m.%Y %H:%M:%S'
dt_start = dt.strptime("22.04.2022 00:00:00", dt_format)
dt_end = dt.strptime("01.09.2022 00:00:00", dt_format)
mask = (df_pw.index >= dt_start) & (df_pw.index < dt_end)
data = df_pw[mask]
df_umenge = pd.DataFrame(columns=['Beschrieb', 'h [m]', 'Q [l/s]', 'V [m3]',
    'V_cum [m3]'])  
for pw, info in pw_info.items():
    if info['h_entl']:
        data1 = data.loc[data['Beschrieb'].str.contains(pw)]
        data1 = data1.loc[data1['Beschrieb'].str.contains('Niveau')]
        for i, el in enumerate(data1['Beschrieb'].unique()):
            data2 = data1[data1['Beschrieb']==el]
            data2 = data2[data2['Wert']>info['h_entl']]
            data2['Wert'] -= info['h_entl']
            data2.rename(columns={'Wert': 'h [m]'}, inplace=True)
            data2['Q [l/s]'] = (Q_poleni(0.6, info['B_entl'], data2['h [m]'])
                * 1e3)
            data2['V [m3]'] = data2['Q [l/s]'] * 60 / 1e3
            data2['V_cum [m3]'] = data2['V [m3]'].cumsum()
            df_umenge = pd.concat((df_umenge, data2))

save_path = save_dir / 'Pumpwerken/Überlaufmengen.csv'
df_umenge.to_csv(save_path, sep=';')

### Pumpenstunden

In [ ]:
h_pws = pd.DataFrame()

for pump in df_betrieb['Beschrieb'].unique():
    data = df_betrieb[df_betrieb['Beschrieb']==pump]
    data = data[data['Wert']==1]
    times = data.index.to_numpy()
    t1 = times[0:-2:2]
    t2 = times[1:-1:2]
    delta_t = (t2 - t1)
    delta_t = pd.DataFrame(delta_t)
    tot_h = delta_t.sum().dt.total_seconds() / 60**2
    h_pws.at[0, pump] = tot_h.values[0]

save_path = save_dir / 'Pumpwerken/Pumpenstunden.csv'
h_pws.sort_index(axis='columns', inplace=True)
h_pws = h_pws.T
h_pws.to_csv(save_path, sep=';')

### Rain

In [ ]:
tot_rain_am = ts_rain_am.sum()
print(tot_rain_am)

tot_rain_ms = ts_rain_ms.sum()
print(tot_rain_ms)

## Plots

In [ ]:
fig = go.Figure()

for name in sorted(df_betrieb['Beschrieb'].unique()):
    x = df_betrieb[df_betrieb['Beschrieb']==name].index
    y = df_betrieb[df_betrieb['Beschrieb']==name]['Wert']
    fig.add_trace(go.Scatter(x=x, y=y, name=name))

save_path = save_dir / 'Pumpwerken/Plots/Pumpenbetrieb.html'
fig.write_html(save_path)

In [ ]:
for pw, info in pw_info.items():
    data = df_pw[df_pw['Beschrieb'].str.contains(pw)]
    
    fig = go.Figure()
    for el in sorted(data['Beschrieb'].unique()):
        print(el)
        x = data[data['Beschrieb']==el].index
        y = data[data['Beschrieb']==el]['Wert']
        fig.add_trace(go.Scatter(x=x, y=y, name=el))

    if info['h_entl']:
        fig.add_hline(y=info['h_entl'], line_width=0.5)
    fig.update_xaxes(title='Datum')
    fig.update_yaxes(title='Niveau [m]')
    fig.update_layout(title=f'{pw}')
    save_path = save_dir / f'Pumpwerken/Plots/{pw}.html'
    fig.write_html(save_path)

#### Rain

In [ ]:
fig = go.Figure()
x = ts_rain_ms.index
y1 = ts_rain_ms.iloc[:,0]
fig.add_trace(go.Scatter(x=x, y=y1, yaxis='y1'))

y2 = ts_rain_ms.iloc[:,0].cumsum()
fig.add_trace(go.Scatter(x=x, y=y2, yaxis='y2'))

save_path = save_dir / f'Regendaten/Plots/Rain_ms.html'
fig.write_html(save_path)

In [ ]:
max(y1)